In [1]:
import cv2
import numpy as np
import torch
from torchvision import models
import torchvision.transforms as T
from PIL import Image


def decode_segmap(image, nc=21):
   
  label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (255, 255, 255),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])
 
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
   
  for l in (0,15):
    idx = image == l
    r[idx] = label_colors[l, 0]
    g[idx] = label_colors[l, 1]
    b[idx] = label_colors[l, 2]
     
  rgb = np.stack([r, g, b], axis=2)
  return rgb

In [4]:

#fcn = models.segmentation.fcn_resnet101(pretrained=True).eval().cuda()
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval().cuda()
transform = T.Compose([T.ToTensor(), 
                       T.Normalize(mean = [0.485, 0.456, 0.406], 
                                   std = [0.229, 0.224, 0.225])])

In [5]:
vc = cv2.VideoCapture('/home/juancamilog/Downloads/hellou.mov')

In [6]:
ret, img = vc.read()
while ret:
    img = cv2.pyrDown(img)
    im_pil = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(im_pil)
    inp = transform(im_pil).unsqueeze(0).cuda()
    #inp = 
    with torch.no_grad():
        out = dlab(inp)['out']
        torch.cuda.empty_cache()
    om = torch.argmax(out.squeeze(), dim=0).detach().cpu().numpy()
    rgb = decode_segmap(om)
    cv2.imshow("original", img)
    mask = np.zeros((540,960, 3)).astype(np.uint8)
    mask[:,:] = rgb
    mask = cv2.resize(mask, (img.shape[1],img.shape[0]))
    masked_img = img.copy()
    masked_img[mask == 0] = 0
    cv2.imshow("segmented", masked_img)
    c = cv2.waitKey(1)
    if c == ord('x'):
        break
    ret, img = vc.read()